# Load the data

In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.single_click_ent import sce_BK as pbk; reload(pbk)
from analysis.lib.single_click_ent import Analysis_params_BK as BK_params;reload(BK_params)

%matplotlib inline

folder_lt4 = r'D:\measuring\data'
folder_lt3 = r'Z:\data'

### ssro calibrations
day = '20170523_' # for ssro calibration

ssro_calib_lt4 = day+'172325'
ssro_calib_lt3 = day+'082327'

BK_analysis = pbk.BK_analysis('XX',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
# a = ppq.purify_pq(folder, hdf5_mode='r')#, pq_folder='bs_remote')
# b = ppq.purify_pq(lt3_folder, hdf5_mode='r')#, pq_folder='bs_remote')

In [ ]:
### list of good timestamps and RO calibration
all_lt3,all_lt4 = [],[]

### get all valid time stamps. remember that data has to be stored locally.
msmt_days  = ['20170524']
newest_timestamps = ['140000']
oldest_timestamps = ['121640']

for d, newest_timestamp, oldest_timestamp in zip(msmt_days,newest_timestamps,oldest_timestamps):

    tstamp_lt3,tstamp_lt4 = BK_analysis.tstamps_for_both_setups(d,newest_tstamp = newest_timestamp, oldest_tstamp = oldest_timestamp) ### newest timestamp allows for only taking parts of a day.
    all_lt3.extend(tstamp_lt3)
    all_lt4.extend(tstamp_lt4)


In [ ]:
BK_analysis.load_raw_data(all_lt3,all_lt4)
# BK_analysis.filter_CR_after(threshold =None)

In [ ]:
# BK_analysis.check_tail_w1_w2()

In [ ]:
### autocorrelation of photon clicks --> do we have huge waiting times in the experiment
# reload(pq_plots)
# a_lt4 = ppq.purifyPQAnalysis(tb.latest_data(tstamp_lt4[0],folder = folder_lt4))
# print a_lt4
# pq_plots.plot_autocorrelation_histogram(a_lt4.pqf,start = 0,length = 200e3,binsize=0.1e3)

### Filtering and correlations

In [ ]:
### this window can be used to analyze one of the full timetrace in order to get a feeling for the tail and the windows.
### later on all timestamps should be combined to get a clear pikkcture
reload(BK_params)
a_lt4 = ppq.purifyPQAnalysis(tb.latest_data(all_lt4[0],folder = folder_lt4))

vert0 = BK_params.BK_params['st_start']/1000. ### start of the first window
vert1 = (BK_params.BK_params['st_start']+BK_params.BK_params['ch1_offset'])/1000. ### start of the first window

# BK_analysis.plot_total_histogram(1,st_start-20e3,200e3,1e3,log_plot=True)
pq_plots.plot_marker_filter_comparison(a_lt4.pqf,log= True, mrkr_chan = 1,
                                       start =BK_params.BK_params['st_start']-7e3,
                                       length = BK_params.BK_params['st_len']+10e3+BK_params.BK_params['ch1_offset'],
                                       hist_binsize = 1e2, save = False,
                                       plot_threshold_ch0 =vert0,plot_threshold_ch1 =vert1) #start =7480

vert0 = (BK_params.BK_params['st_start']+BK_params.BK_params['p_sep'])/1000. ### start of the first window
vert1 = (BK_params.BK_params['st_start']+BK_params.BK_params['p_sep']+BK_params.BK_params['ch1_offset'])/1000. ### start of the first window

# BK_analysis.plot_total_histogram(1,st_start-20e3,200e3,1e3,log_plot=True)
pq_plots.plot_marker_filter_comparison(a_lt4.pqf,log= True, mrkr_chan = 1,
                                       start =BK_params.BK_params['st_start']+BK_params.BK_params['p_sep']-7e3,
                                       length = BK_params.BK_params['st_len']+10e3+BK_params.BK_params['ch1_offset'],
                                       hist_binsize = 1e2, save = False,
                                       plot_threshold_ch0 =vert0,plot_threshold_ch1 =vert1) #start =7480

In [ ]:
### return the length of events without temporal filter:
reload(BK_params)

no_of_events = 0
for i in range(len(BK_analysis.lt3_dict['ssro_results'])):
    no_of_events += len(BK_analysis.lt3_dict['ssro_results'][i])

print 'number of unfiltered (temporal and CR) events', no_of_events



### apply temporal filter (see above for filters)
BK_analysis.filter_CR_after(threshold = 1,verbose = False)
BK_analysis.apply_temporal_filters_to_prefiltered_data()


In [ ]:
### filter the PQ data: Return an array which is True at each position where an event was in a window
BK_analysis.attach_state_filtered_syncs(verbose = False)

In [ ]:
### applies the time fitlered sync numbers to the adwin RO results of each setup
BK_analysis.correlate_RO_results(verbose = True,apply_ROC = True)

In [ ]:
### correlations vs. window start?
st_start = BK_params.BK_params['st_start']
BK_analysis.sweep_filter_parameter_vs_correlations('st_start',st_start + np.linspace(-4e3,5e3,8))

In [ ]:
BK_analysis.sweep_filter_parameter_vs_correlations('st_len',np.linspace(2e3,40e3,10))

In [ ]:
BK_analysis.sweep_filter_parameter_vs_correlations('ch1_offset', BK_params.BK_params['ch1_offset'] + np.linspace(-5e3,5e3,25))

In [ ]:
BK_analysis.sweep_filter_parameter_vs_correlations('dt_max',np.linspace(2e3,30e3,15))

In [ ]:
BK_analysis.sweep_filter_parameter_vs_correlations('CR_after_threshold',np.linspace(1,50,7))